# Introduction

NexDatawork is an AI data agent for data engineering and analytics without writing code.

## Prerequisites

 - Node.js
 - Supabase
 - OpenAI


 Before starting your work install all the required tools:

In [ ]:
!pip install langchain langchain-openai langchain-anthropic langchain-google-vertexai
!pip install --upgrade --quiet  langchain-community

Import all the required libraries

In [ ]:
import os
from IPython.display import Markdown, HTML, display
from langchain_openai import AzureChatOpenAI
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.agents.agent_types import AgentType
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent
import gradio as gr
import pandas as pd
import io
import contextlib
from sqlalchemy import (Engine,create_engine,MetaData,Table,Column,String,Integer,Float,insert,inspect,text)
from langchain_core.messages import HumanMessage
from langgraph.prebuilt import create_react_agent
from langchain_community.agent_toolkits import SQLDatabaseToolkit
from langchain.chat_models import init_chat_model
from langchain_community.utilities import SQLDatabase



To access AzureOpenAI models you'll need to create an Azure account, create a deployment of an Azure OpenAI model, get the name and endpoint for your deployment, get an Azure OpenAI API key and install the langchain-openai integration package.

Replace the placeholders with the actual values.

In [ ]:
os.environ["AZURE_OPENAI_ENDPOINT"] = "INSERT THE AZURE OPENAI ENDPOINT"
os.environ["AZURE_OPENAI_API_KEY"] = "INSERT YOUR AZURE OPENAI API KEY"
os.environ["OPENAI_API_KEY"] = "INSERT YOUR OPENAI API KEY"

To set up the Azure OpenAI model choose the name for ```AZURE_DEPLOYMENT_NAME``` and insert ```AZURE_API_VERSION``` (the latest supported version can be found here: https://learn.microsoft.com/en-us/azure/ai-services/openai/reference).

In [ ]:
# Load your Azure environment variables
AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
AZURE_DEPLOYMENT_NAME = "gpt-4.1"  # 👈 Change if needed
AZURE_API_VERSION = "2025-01-01-preview"  # 👈 Use your correct version

# Define Azure LLM with streaming enabled
model = AzureChatOpenAI(
    openai_api_version=AZURE_API_VERSION,
    azure_deployment=AZURE_DEPLOYMENT_NAME,
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
)


Here you can create a prompt for your model.

In [ ]:
# Prompt
CSV_PROMPT_PREFIX = """
Set pandas to show all columns.
Get the column names and infer data types.
Then attempt to answer the question using multiple methods.
Please provide only the Python code required to perform the action, and nothing else.
"""

CSV_PROMPT_SUFFIX = """
- Try at least 2 different methods of calculation or filtering.
- Reflect: Do they give the same result?
- After performing all necessary actions and analysis with the dataframe, return the answer in clean **Markdown**, include summary table if needed.
- Include **Execution Recommendation** and **Web Insight** in the final Markdown.
- Always conclude the final Markdown with:

### Final Answer

Your conclusion here.

---

### Explanation

Mention specific columns you used.
Please provide only the Python code required to perform the action, and nothing else until the final Markdown output.
"""


#prompt for creating SQL queries
system_message = """
You are an agent designed to interact with a SQL database.
Given an input question, create a syntactically correct {dialect} query to run,
then look at the results of the query and return the answer. Unless the user
specifies a specific number of examples they wish to obtain, always limit your
query to at most {top_k} results.

You can order the results by a relevant column to return the most interesting
examples in the database. Never query for all the columns from a specific table,
only ask for the relevant columns given the question.

You MUST double check your query before executing it. If you get an error while
executing a query, rewrite the query and try again.

DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the
database.

To start you should ALWAYS look at the tables in the database to see what you
can query. Do NOT skip this step.

Then you should query the schema of the most relevant tables.
""".format(
    dialect="SQLite",
    top_k=5,
)


The following block is responsible for the logic of the agent and the output that it produces.

In [ ]:
# Replace this with your actual LLM setup
# Example:
# from langchain_openai import AzureChatOpenAI
# model = AzureChatOpenAI(...)

def sql_code(files, question):
    try:
        dfs = [pd.read_csv(f.name) for f in files]
    except Exception as e:
        return f"❌ Could not read CSVs: {e}", ""
    try:
        agent = create_pandas_dataframe_agent(
        llm=model,
        df=df,
        verbose=True,
        agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
        allow_dangerous_code=True,
        handle_parsing_errors=True,  # 👈 this is the fix
      )




# --- Agent Logic ---
def ask_agent(files, question):
    try:
        dfs = [pd.read_csv(f.name) for f in files]
        df = pd.concat(dfs, ignore_index=True)
    except Exception as e:
        return f"❌ Could not read CSVs: {e}", ""

    try:
        agent = create_pandas_dataframe_agent(
        llm=model,
        df=df,
        verbose=True,
        agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
        allow_dangerous_code=True,
        handle_parsing_errors=True,  # 👈 this is the fix
    )


        full_prompt = CSV_PROMPT_PREFIX + question + CSV_PROMPT_SUFFIX

        buffer = io.StringIO()
        with contextlib.redirect_stdout(buffer):
            result = agent.invoke(full_prompt)
        trace = buffer.getvalue()
        output = result["output"]


        return output, trace

    except Exception as e:
        return f"❌ Agent error: {e}", ""


The block below deals with creating SQL code.

In [ ]:
#function create_db receives a dictionary with table names as a key and tables as values
def create_db(files):
    engine = create_engine("sqlite:///database.db")
    with engine.begin() as connection:
      for name,table in zip(files.keys(),files.values()):
        table.to_sql(name,connection,if_exists="replace",index=False)

    db = SQLDatabase.from_uri("sqlite:///database.db")
    return db

In [ ]:
#Initialization of a LLM model for SQL queries
def start_llm(database):
    model = init_chat_model("gpt-4o-mini", model_provider="openai")

    toolkit = SQLDatabaseToolkit(db=database, llm=model)

    tools = toolkit.get_tools()
    return model, tools


def extract_code(HumanMessage):
    FRONT_INDENT = len('```sql\n')
    BACK_INDENT = len('\n')
    p1 = HumanMessage.find('`')
    p2 = p1+FRONT_INDENT
    while HumanMessage[p2]!='`':
      p2+=1
    return HumanMessage[p1+FRONT_INDENT:p2-BACK_INDENT]

#Function that receives dataframes, puts them in a database and uses an AI agent to create quieries based on the user's question
def pipeline(tables,question):
    db = create_db(tables)
    llm, tools = start_llm(db)
    agent_executor = create_react_agent(llm, tools, prompt=system_message)
    output = ""
    for step in agent_executor.stream(
        {"messages": [{"role": "user", "content": question}]},
        stream_mode="values",
    ):
        output += step["messages"][-1].content
    query = extract_code(output)
    return query

The next section creates the GUI and starts the programme.

In [ ]:
# --- Gradio UI ---
with gr.Blocks(
    css="""
    body, .gradio-container {
        background: #ffffff !important;
        color: #1f2937 !important;
        font-family: 'Segoe UI', sans-serif;
    }

    #title {
        color: #1f2937 !important;
        font-size: 2rem;
        font-weight: 600;
        text-align: center;
        padding-top: 20px;
        padding-bottom: 10px;
    }

    .gr-box, .gr-input, .gr-output, .gr-markdown, .gr-textbox, .gr-file, textarea, input {
        background: rgba(0, 0, 0, 0.04) !important;
        border: 1px solid rgba(0, 0, 0, 0.1);
        border-radius: 12px !important;
        color: #1f2937 !important;
    }

    textarea::placeholder, input::placeholder {
        color: rgba(31, 41, 55, 0.6) !important;
    }

    button {
        background: rgba(0, 0, 0, 0.07) !important;
        color: #1f2937 !important;
        border: 1px solid rgba(0, 0, 0, 0.15) !important;
        border-radius: 8px !important;
    }

    button:hover {
        background: rgba(0, 0, 0, 0.15) !important;
    }
    """
) as demo:

    gr.Markdown("<h2 id='title'>📊 NexDatawork Data Agent</h2>")

    with gr.Column():
        result_display = gr.Markdown(label="📌 Report Output (Markdown)")
        trace_display = gr.Textbox(label="🛠️ Data Agent Reasoning - Your Explainable Agent", lines=20)

    with gr.Row(equal_height=True):
        file_input = gr.File(label="📁 Upload CSV(s)", file_types=[".csv"], file_count="multiple")
        with gr.Column():
            question_input = gr.Textbox(
        label="💬 Ask Your Data",
        placeholder="e.g., What is the trend for revenue over time?",
        lines=2
    )
            sql_code = gr.Interface(fn=,label="SQL code",placeholder='SQL code will appear here',lines=2)


    ask_button = gr.Button("💡 Analyze")

    ask_button.click(
        fn=ask_agent,
        inputs=[file_input, question_input],
        outputs=[result_display, trace_display]
    )

demo.launch(share=True)